## Semana 02 - Big Data

## Bibliotecas

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

## Setup

In [2]:
import findspark
findspark.init()

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
df = spark.createDataFrame([('Pedro', '4'), ('João', '5')], schema = 'nome STRING, id STRING')

In [5]:
df.show()

+-----+---+
| nome| id|
+-----+---+
|Pedro|  4|
| João|  5|
+-----+---+



## Acessando os Tipos de Dados

In [13]:
from pyspark.sql.types import *



In [7]:
int_type = IntegerType()
int_type


IntegerType

In [8]:
array_type = ArrayType(IntegerType())
array_type

ArrayType(IntegerType,true)

## Convertendo os tipos de Colunas

In [9]:
df.dtypes

[('nome', 'string'), ('id', 'string')]

In [10]:
df.select('nome', col('id').cast(IntegerType())).dtypes

[('nome', 'string'), ('id', 'int')]

### forma alternativa
IntegerType() == 'int'
StringType() == 'string'
FloatType() == 'float'

ArrayType(IntegralType()) == 'ARRAY<INT>'

In [16]:
df.select('nome', col('id').cast('int')).dtypes


[('nome', 'string'), ('id', 'int')]

## Schema e Criação de DataFrames
Um schema no Spark é uma especificação de tipos das colunas de um DataFrame. Eles são usados na leitura de dados externos e criação de DataFrames, e podem ser passados diretamente ao Spark ou podem ser inferidos. Passar um schema na leitura traz benefícios interessantes, como:

Evita que o Spark faça inferência de tipos, o que é custoso e demorado dependendo do tamanho do arquivo, além de propenso a erros;
Permite que o usuário identifique erros nos dados logo na leitura, caso os dados não sigam o schema especificado.

In [17]:

df = spark.createDataFrame([('Pedro', 1), ('João', 6), ('Juliana', 4), ('Lucas', 7)], schema = ['nome', 'id'])

In [18]:
df.show()

+-------+---+
|   nome| id|
+-------+---+
|  Pedro|  1|
|   João|  6|
|Juliana|  4|
|  Lucas|  7|
+-------+---+



In [19]:
df.dtypes

[('nome', 'string'), ('id', 'bigint')]

## Criando Schemas programaticamente

In [20]:
schema = StructType([
    StructField('nome', StringType()),
    StructField('id', IntegerType())
])

In [21]:
df = spark.createDataFrame([('Pedro', 1), ('João', 6), ('Juliana', 4), ('Lucas', 7)], schema = schema)

In [22]:
df.dtypes

[('nome', 'string'), ('id', 'int')]

## Criando schemas com DDL

In [23]:
schema = 'nome STRING, id INT'


In [24]:
df = spark.createDataFrame([('Pedro', 1), ('João', 6), ('Juliana', 4), ('Lucas', 7)], schema = schema)

In [25]:
df.dtypes

[('nome', 'string'), ('id', 'int')]

## Criando DataFrames

In [26]:
data = [('Pedro', 1), ('João', 6), ('Juliana', 4), ('Lucas', 7)]

In [27]:
schema = 'nome STRING, id INT'


In [28]:
df = spark.createDataFrame(data, schema=schema)

In [29]:
df.dtypes

[('nome', 'string'), ('id', 'int')]

In [30]:
df.schema


StructType(List(StructField(nome,StringType,true),StructField(id,IntegerType,true)))

In [31]:
df.printSchema()

root
 |-- nome: string (nullable = true)
 |-- id: integer (nullable = true)



In [32]:

spark.range(100).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+
only showing top 20 rows



In [ ]:
#df = spark.createDataFrame(pandas_df)

## Leitura e Escrita de Dados

### DataFrameReader
spark.read.format(format).option(args).load(file/path)

### DataFrameWriter
df.write.format(format).option(args).save(file/path)
Lendo e Escrevendo CSV
Opções mais comuns:

<li>header</li>
<li>inferSchema</li>
<li>sep</li>
<li>encoding</li>

In [39]:
data_path = './'

In [40]:
file_path = 'df_cnae.csv'

In [41]:
df = spark.read.format('csv').load(file_path)

In [43]:
df.limit(15).show()


+--------------------+
|                 _c0|
+--------------------+
|"0111301";"Cultiv...|
|"0111302";"Cultiv...|
|"0111303";"Cultiv...|
|"0111399";"Cultiv...|
|"0112101";"Cultiv...|
|"0112102";"Cultiv...|
|"0112199";"Cultiv...|
|"0113000";"Cultiv...|
|"0114800";"Cultiv...|
|"0115600";"Cultiv...|
|"0116401";"Cultiv...|
|"0116402";"Cultiv...|
|"0116403";"Cultiv...|
|"0116499";"Cultiv...|
|"0119901";"Cultiv...|
+--------------------+



## Definindo o schema¶

In [44]:
schema = 'cod_cnae STRING, descricao_cnae STRING'


## Opção de Correção 1

In [46]:
df = spark.read.csv(file_path, sep=';', encoding='ISO-8859-1', schema=schema)
df.limit(5).show()

+--------+--------------------+
|cod_cnae|      descricao_cnae|
+--------+--------------------+
| 0111301|    Cultivo de arroz|
| 0111302|    Cultivo de milho|
| 0111303|    Cultivo de trigo|
| 0111399|Cultivo de outros...|
| 0112101|Cultivo de algodÃ...|
+--------+--------------------+



## Opção de Correção 2¶

In [47]:
df = (
    spark.read
    .format('csv')
    .option('sep', ';')
    .option('encoding', 'ISO-8859-1')
    .schema(schema)
    .load(file_path)
)
df.limit(5).show()

+--------+--------------------+
|cod_cnae|      descricao_cnae|
+--------+--------------------+
| 0111301|    Cultivo de arroz|
| 0111302|    Cultivo de milho|
| 0111303|    Cultivo de trigo|
| 0111399|Cultivo de outros...|
| 0112101|Cultivo de algodÃ...|
+--------+--------------------+



## Opção de Correção 3

In [48]:
df = (
    spark.read
    .format('csv')
    .options(sep=';', encoding='ISO-8859-1')
    .schema(schema)
    .load(file_path)
)
df.limit(5).show()


+--------+--------------------+
|cod_cnae|      descricao_cnae|
+--------+--------------------+
| 0111301|    Cultivo de arroz|
| 0111302|    Cultivo de milho|
| 0111303|    Cultivo de trigo|
| 0111399|Cultivo de outros...|
| 0112101|Cultivo de algodÃ...|
+--------+--------------------+



Obs: utilizando o método "options" podemos parametrizar melhor nossa função usando um dicionário


In [49]:
options_dict = {
    'sep': ';',
    'encoding': 'ISO-8859-1',
}

df = (
    spark.read
    .format('csv')
    .options(**options_dict)
    .schema('cod_cnae INT, descricao_cnae STRING')
    .load(file_path)
)
df.limit(5).show()

+--------+--------------------+
|cod_cnae|      descricao_cnae|
+--------+--------------------+
|  111301|    Cultivo de arroz|
|  111302|    Cultivo de milho|
|  111303|    Cultivo de trigo|
|  111399|Cultivo de outros...|
|  112101|Cultivo de algodÃ...|
+--------+--------------------+

